In [1]:
using Revise
using KLIEPInference
using Distributions
using MathOptInterfaceMosek, SCS
import JuMP

┌ Info: Recompiling stale cache file /home/mkolar/.julia/compiled/v0.7/KLIEPInference/rX42g.ji for KLIEPInference [2424ee94-ddf3-11e8-1bca-091389175f78]
└ @ Base loading.jl:1185


In [18]:
θ = zeros(5, 5)
spl = IsingSampler(θ)
X = rand(spl, 500)
Y = rand(spl, 1000)
Ψx = Ψising(X)
Ψy = Ψising(Y);

10×1000 Array{Float64,2}:
 -1.0  1.0  -1.0  1.0   1.0   1.0   1.0  …   1.0  -1.0   1.0  -1.0  1.0  -1.0
 -1.0  1.0   1.0  1.0  -1.0   1.0   1.0      1.0  -1.0  -1.0   1.0  1.0   1.0
  1.0  1.0   1.0  1.0  -1.0  -1.0   1.0     -1.0   1.0   1.0  -1.0  1.0   1.0
  1.0  1.0   1.0  1.0  -1.0   1.0  -1.0     -1.0   1.0   1.0  -1.0  1.0   1.0
  1.0  1.0  -1.0  1.0  -1.0   1.0   1.0      1.0   1.0  -1.0  -1.0  1.0  -1.0
 -1.0  1.0  -1.0  1.0  -1.0  -1.0   1.0  …  -1.0  -1.0   1.0   1.0  1.0  -1.0
 -1.0  1.0  -1.0  1.0  -1.0   1.0  -1.0     -1.0  -1.0   1.0   1.0  1.0  -1.0
 -1.0  1.0   1.0  1.0   1.0  -1.0   1.0     -1.0  -1.0  -1.0  -1.0  1.0   1.0
 -1.0  1.0   1.0  1.0   1.0   1.0  -1.0     -1.0  -1.0  -1.0  -1.0  1.0   1.0
  1.0  1.0   1.0  1.0   1.0  -1.0  -1.0      1.0   1.0   1.0   1.0  1.0   1.0

In [19]:
#solver = JuMP.with_optimizer(MosekOptimizer, QUIET=false)
solver = JuMP.with_optimizer(SCS.Optimizer, verbose=1)
θhat = KLIEP(Ψx, Ψy, solver, SOCP_KLIEP())

  3.425239 seconds (281.22 k allocations: 29.979 MiB, 0.51% gc time)
----------------------------------------------------------------------------
	SCS v2.0.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 13000, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 1011, constraints m = 3001
Cones:	linear vars: 1
	exp vars: 3000, dual exp vars: 0
Setup time: 1.01e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 1.09e+00  5.20e-01  5.23e-02 -1.51e+01 -1.35e+01  2.24e-15  2.34e-02 
   100| 1.26e-02  3.01e-03  1.99e-03 -7.89e-02 -8.12e-02  3.41e-15  

10-element Array{Float64,1}:
 -0.10732113223697365 
  0.015635747665701284
 -0.033053198134818905
  0.04949717722833603 
 -0.02806154004687123 
  0.0151376788410599  
  0.0847280435482009  
 -0.12337812196207168 
 -0.035903668493558714
 -0.04134721351828584 

In [24]:
θx = zeros(5, 5)
splx = IsingSampler(θx)
X = rand(splx, 500)

θy = zeros(5, 5)
θy[1, 2] = .3
θy[2, 1] = .3
sply = IsingSampler(θy)
Y = rand(sply, 1000)

Ψx = Ψising(X)
Ψy = Ψising(Y);

In [25]:
#solver = JuMP.with_optimizer(MosekOptimizer, QUIET=false)
solver = JuMP.with_optimizer(SCS.Optimizer, verbose=1)
θhat = KLIEP(Ψx, Ψy, solver, SOCP_KLIEP())

----------------------------------------------------------------------------
	SCS v2.0.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 13000, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 1011, constraints m = 3001
Cones:	linear vars: 1
	exp vars: 3000, dual exp vars: 0
Setup time: 8.33e-04s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 1.13e+00  5.36e-01  6.36e-02 -1.57e+01 -1.38e+01  2.29e-15  3.34e-02 
   100| 9.51e-04  2.30e-04  3.63e-03 -6.67e-02 -6.26e-02  2.23e-15  1.01e+00 
   200| 3.02e-05  7.41e-06  1.11e-04 -6.27e-02 -6.25e-02  3

10-element Array{Float64,1}:
 -0.3159677981021649   
 -0.0025015119153493502
 -0.0004152897812701905
  0.053351047049196895 
 -0.02364891195210766  
 -0.06900210314922718  
  0.09455196922730602  
 -0.03741948788271076  
  0.07268540138485453  
 -0.05221240382551319  

In [47]:
sum(Y, dims=2)

5×1 Array{Int64,2}:
 5114
 5134
 5018
 5075
 4994